In [1]:
!nvidia-smi
from tensorflow.keras import layers
import tensorflow as tf

Mon Mar  4 23:32:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.58                 Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0              19W /  80W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [3]:
#!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [4]:
#unzip_data('nlp_getting_started.zip')

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state = 42)

In [7]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [10]:
len(train_df),len(test_df)

(7613, 3263)

In [11]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target : {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text : \n{text}\n")
    print("----\n")
    


Target : 0 (not real disaster)
Text : 
Know them recognize them......then obliterate them! 
#gym #gymflow #gymtime #team #assassinsÛ_ https://t.co/mUHj8CbdQb

----

Target : 1 (real disaster)
Text : 
There might be casualties tomorrow

----

Target : 0 (not real disaster)
Text : 
[Jax(MK2)] Stage Fatality:UUD+LK (Close)

----

Target : 0 (not real disaster)
Text : 
The trouble in one of Buffett's favorite sectors http://t.co/J4dqPFLMkR

----

Target : 0 (not real disaster)
Text : 
If GOP want to destroy America then Obama is dilutional I should be institutionalize or sued for slander.  https://t.co/Z9Zj3KxwYU

----



In [12]:
#007 Splitting data into training and validation sets.mp4
from sklearn.model_selection import train_test_split
train_sentences , val_sentences ,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)
                                                                            
                                                                            

In [13]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [14]:
#008 Converting text data to numbers using tokenisation and embeddings (overview).mp4
#009 Setting up a TensorFlow TextVectorization layer to convert text to numbers.mp4
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#OOV  = "Out of Vocabulary"
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)
                                    
                                    
                                    

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length,
                                   )

In [17]:
#010 Mapping the TextVectorization layer to text data and turning it into numbers
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [18]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 C-130 specially modified to land in a stadium and rescue hostages in Iran in 1980... http://t.co/tNI92fea3u http://t.co/czBaMzq3gL\n
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1544, 1359, 1501,    5,  481,    4,    3, 1358,    7,  389,  422,
           4,  458,    4, 1443]], dtype=int64)>

In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
#[UNK] meand unknown token

In [20]:
len(words_in_vocab)

10000

In [21]:
#011 Creating an Embedding layer to turn tokenised text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )

In [22]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Tomorrow is Internet Day. It has been almost 2 months. I look forward* to the deluge of stuff I've been avoiding.   *a downright lie\n
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04683949, -0.04235332,  0.01471429, ...,  0.01488325,
         -0.0458356 , -0.00430504],
        [-0.00432491, -0.04747291, -0.01937131, ..., -0.03290159,
          0.02881483, -0.03868037],
        [ 0.04001212,  0.00591483,  0.04197618, ...,  0.04208598,
          0.03157042, -0.02523223],
        ...,
        [-0.02164348, -0.01757858, -0.04358907, ..., -0.02692263,
          0.00607443, -0.02353704],
        [ 0.01454177,  0.04026485,  0.0229558 , ..., -0.01801107,
          0.0035781 , -0.01201916],
        [-0.01177815, -0.00803412,  0.04943955, ...,  0.0354838 ,
          0.01396375, -0.04119672]]], dtype=float32)>

In [23]:
#check out a single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04683949, -0.04235332,  0.01471429, -0.02541248,  0.00592002,
        -0.00208192, -0.02438792,  0.01024276,  0.00782311,  0.01977135,
        -0.02586137, -0.04503288,  0.03896813,  0.01469218,  0.04036864,
        -0.03145806, -0.04422717, -0.00201411, -0.03484816, -0.01865953,
        -0.03337889, -0.0126461 ,  0.0455013 ,  0.04249125, -0.01575047,
        -0.0448462 ,  0.02307824,  0.02525781, -0.03799189, -0.00787159,
        -0.03178555, -0.02808758, -0.00846045, -0.00842472,  0.00436361,
        -0.02542552,  0.03922491,  0.02041407,  0.03077209,  0.04580322,
        -0.0337895 ,  0.01835178, -0.02556906,  0.02105394,  0.00818021,
        -0.02542811,  0.03490517, -0.0292873 ,  0.00973149,  0.0403739 ,
         0.04627251, -0.04572434,  0.04136213,  0.04782133,  0.01370401,
         0.00894644, -0.02116194,  0.02874464, -0.00397475, -0.0220012 ,
         0.03691032,  0.00425458,  0.04014914, -0.03942889, -0.04364925,
  

In [24]:
#012 Discussing the various modelling experiments we're going to run
#013 Model 0 Building a baseline model to try and improve upon
#Multinomial naive bais using TF_IDF formula
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),

])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
#Evalute our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of : 79.27%


In [26]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [27]:
#014 Creating a function to track and evaluate our model's results
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall":model_recall,
                     "f1":model_f1}
    return model_results

In [28]:
baseline_results = calculate_results(y_true=val_labels,y_pred=baseline_preds)
baseline_results
#higher precision leads to less false positives
#higher recall leads to less false negetives

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [29]:
#015 Model 1 Building, fitting and evaluating our first deep model on text data
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers
SAVE_DIR = "model_logs"
#build model with Function API
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) #layers.Reshape((1920,))(x) #.
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")
model_1.summary()



Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [30]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240304-233245
Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.6111 - accuracy: 0.6962 - val_loss: 0.5362 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4420 - accuracy: 0.8164 - val_loss: 0.4718 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3469 - accuracy: 0.8631 - val_loss: 0.4599 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2844 - accuracy: 0.8914 - val_loss: 0.4656 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2381 - accuracy: 0.9102 - val_loss: 0.4857 - val_accuracy: 0.7913


In [31]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4857 - accuracy: 0.7913


[0.48570823669433594, 0.7913385629653931]

In [32]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

24/24 [==============================] - 0s 1ms/step


array([0.35078928], dtype=float32)

In [33]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [34]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [35]:
#calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.13385826771653,
 'precision': 0.7986313296680336,
 'recall': 0.7913385826771654,
 'f1': 0.7877263030663991}

In [36]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [37]:
#016 Visualising our model's learned word embeddings with TensorFlow's projector tool
words_in_vocab = text_vectorizer.get_vocabulary()

In [38]:
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape
#https://projector.tensorflow.org/
#https://www.tensorflow.org/text/guide/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk

(10000, 128)

In [41]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [42]:
#017 High-level overview of Recurrent Neural Networks (RNNs) + where to learn more
#018 Model 2 Building, fitting and evaluating our first TensorFlow RNN model (LSTM)
#MOdel 2 LSTM
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")


In [43]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [44]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240304-233252
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.2258 - accuracy: 0.9215 - val_loss: 0.5954 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1554 - accuracy: 0.9416 - val_loss: 0.7845 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1287 - accuracy: 0.9514 - val_loss: 0.6445 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1113 - accuracy: 0.9600 - val_loss: 0.8368 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0858 - accuracy: 0.9666 - val_loss: 0.8562 - val_accuracy: 0.7743


In [45]:
#make prediction with LSTM Model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.9441582e-02],
       [9.0441984e-01],
       [9.9973363e-01],
       [5.1999837e-02],
       [7.1415870e-04],
       [9.9781954e-01],
       [4.9461260e-01],
       [9.9979931e-01],
       [9.9967480e-01],
       [4.0388736e-01]], dtype=float32)

In [46]:
model_2_pred=tf.squeeze(tf.round(model_2_pred_probs))

In [47]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred)

In [48]:
model_2_results

{'accuracy': 77.42782152230971,
 'precision': 0.7765437107187567,
 'recall': 0.7742782152230971,
 'f1': 0.7719700099160278}

In [49]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [50]:
#019 Model 3 Building, fitting and evaluating a GRU-cell powered RNN
#Gated Recurrent Unit
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x= layers.LSTM(64,return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x= layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [51]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                       

In [52]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240304-233302
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.1538 - accuracy: 0.9434 - val_loss: 0.7698 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0856 - accuracy: 0.9682 - val_loss: 0.7333 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0739 - accuracy: 0.9726 - val_loss: 0.9789 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0599 - accuracy: 0.9746 - val_loss: 0.9493 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0543 - accuracy: 0.9783 - val_loss: 1.1334 - val_accuracy: 0.7625


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_pred)
model_3_results

24/24 [==============================] - 0s 2ms/step


{'accuracy': 76.24671916010499,
 'precision': 0.7657677169674478,
 'recall': 0.7624671916010499,
 'f1': 0.7593667569927582}

In [54]:
model_2_results

{'accuracy': 77.42782152230971,
 'precision': 0.7765437107187567,
 'recall': 0.7742782152230971,
 'f1': 0.7719700099160278}

In [55]:
#020 Model 4 Building, fitting and evaluating a bidirectional RNN model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x= layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [56]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [57]:
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])
            

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240304-233312
Epoch 1/5
215/215 [==============================] - 8s 19ms/step - loss: 0.1013 - accuracy: 0.9661 - val_loss: 0.8899 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0568 - accuracy: 0.9759 - val_loss: 1.1770 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0470 - accuracy: 0.9783 - val_loss: 1.3058 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0411 - accuracy: 0.9796 - val_loss: 1.6611 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0435 - accuracy: 0.9794 - val_loss: 1.4433 - val_accuracy: 0.7638


In [58]:
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 1s 6ms/step


In [59]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [60]:
model_4_results = calculate_results(y_true=val_labels,y_pred = model_4_preds)

In [61]:
model_4_results

{'accuracy': 76.37795275590551,
 'precision': 0.7666261988309462,
 'recall': 0.7637795275590551,
 'f1': 0.7609269783019205}

In [62]:
model_3_results

{'accuracy': 76.24671916010499,
 'precision': 0.7657677169674478,
 'recall': 0.7624671916010499,
 'f1': 0.7593667569927582}

In [63]:
#021 Discussing the intuition behind Conv1D neural networks for text and sequences
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters = 64,
                        kernel_size = 5,
                        activation="relu",
                        padding = "same")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape ,conv_1d_output.shape , max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 64]), TensorShape([1, 64]))

In [64]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0430529 , -0.03901882,  0.0316437 , ..., -0.00072285,
         -0.01119792,  0.04391022],
        [-0.04272015, -0.08692022,  0.00799588, ..., -0.08043483,
          0.04321388, -0.02474188],
        [-0.03645169,  0.0246727 ,  0.03322756, ..., -0.03470501,
          0.00606528,  0.04534715],
        ...,
        [-0.00921816,  0.02901071,  0.02582885, ...,  0.02468948,
          0.02022912, -0.00686842],
        [-0.00921816,  0.02901071,  0.02582885, ...,  0.02468948,
          0.02022912, -0.00686842],
        [-0.00921816,  0.02901071,  0.02582885, ...,  0.02468948,
          0.02022912, -0.00686842]]], dtype=float32)>

In [65]:
max_pool_output

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.05864724, 0.02735057, 0.07045551, 0.        , 0.0816155 ,
        0.10230835, 0.06987156, 0.01618504, 0.06262296, 0.07144142,
        0.02633143, 0.04911626, 0.06168499, 0.02185782, 0.05735552,
        0.0387727 , 0.07117714, 0.04475717, 0.01665567, 0.03150859,
        0.06367635, 0.04802778, 0.04407874, 0.124891  , 0.02339523,
        0.05642576, 0.03922344, 0.05373288, 0.07125469, 0.03719357,
        0.0685453 , 0.06959   , 0.01814985, 0.03258365, 0.03813126,
        0.03303491, 0.08487967, 0.075492  , 0.05235604, 0.04890801,
        0.07785189, 0.07060143, 0.07457149, 0.07268658, 0.07658042,
        0.09269896, 0.04065049, 0.        , 0.04251404, 0.00547775,
        0.12105596, 0.05244166, 0.07268988, 0.02965585, 0.01785791,
        0.0491086 , 0.07033242, 0.07265531, 0.00698978, 0.03883111,
        0.04617395, 0.04255324, 0.03969531, 0.        ]], dtype=float32)>

In [66]:
#022 Model 5 Building, fitting and evaluating a 1D CNN for text
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size=5,strides = 1,activation = "relu",padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

model_5.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [67]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [68]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20240304-233334
Epoch 1/5
215/215 [==============================] - 2s 6ms/step - loss: 0.1182 - accuracy: 0.9629 - val_loss: 0.8799 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0739 - accuracy: 0.9750 - val_loss: 1.0130 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0602 - accuracy: 0.9774 - val_loss: 1.1240 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0534 - accuracy: 0.9780 - val_loss: 1.1814 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0501 - accuracy: 0.9790 - val_loss: 1.2127 - val_accuracy: 0.7572


In [69]:
model_5_pred_probs = model_5.predict(val_sentences)

24/24 [==============================] - 0s 1ms/step


In [70]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [71]:
model_5_results = calculate_results(y_true = val_labels,
                                    y_pred = model_5_preds,)
model_5_results

{'accuracy': 75.7217847769029,
 'precision': 0.7579616566971975,
 'recall': 0.7572178477690289,
 'f1': 0.7553466343855008}

In [72]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [73]:
#023 Using TensorFlow Hub for pretrained word embeddings (transfer learning for NLP)
#Universal sentence encoder
sample_sentence = "There's a flood in my street!"
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")
with tf.device('/CPU:0'):
    embed_samples = embed([sample_sentence, "When you call the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples)


tf.Tensor(
[[-0.01157025  0.02485911  0.02878049 ... -0.00186125  0.02315821
  -0.01485022]
 [ 0.0359669  -0.08579468 -0.01152743 ... -0.03414336  0.02816025
  -0.00878943]], shape=(2, 512), dtype=float32)


In [76]:
#024 Model 6 Building, training and evaluating a transfer learning model for NLP
# create a keras layer using the USE pretrained layer from tensorflow hub
with tf.device('/CPU:0'):
    sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",input_shape = [],dtype = tf.string,trainable=False,name = "USE")

In [80]:
# create model using the sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(1,activation="sigmoid")
],name = "model_6_USE")

model_6.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [81]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 1)                 513       
                                                                 
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________


In [83]:
#Train a classifier on top of USE pretrained embeddings
with tf.device('/CPU:0'):
    model_6_history = model_6.fit(train_sentences,
                                  train_labels,
                                  epochs=5,
                                  validation_data=(val_sentences,val_labels),
                                  callbacks=[create_tensorboard_callback(SAVE_DIR,"tf_hub_sentence_encoder_USE")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_USE/20240304-235011
Epoch 1/5
215/215 [==============================] - 2s 8ms/step - loss: 0.6479 - accuracy: 0.7392 - val_loss: 0.6132 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.5806 - accuracy: 0.7946 - val_loss: 0.5635 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.5378 - accuracy: 0.7984 - val_loss: 0.5320 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.5095 - accuracy: 0.7971 - val_loss: 0.5114 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4895 - accuracy: 0.8019 - val_loss: 0.4967 - val_accuracy: 0.7887


In [85]:
with tf.device('/CPU:0'):
    model_6_pred_probs = model_6.predict(val_sentences)
    model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
    model_6_results = calculate_results(y_true=val_labels,y_pred=model_6_preds)


24/24 [==============================] - 0s 5ms/step


In [86]:
model_6_results

{'accuracy': 78.87139107611549,
 'precision': 0.7891485217486439,
 'recall': 0.7887139107611548,
 'f1': 0.7876016937745534}

In [87]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
#025 Preparing subsets of data for model 7 (same as model 6 but 10% of data)